In [ ]:
import numpy as np
from nltk.corpus import brown
import nltk

# Đảm bảo đã tải bộ dữ liệu brown và universal tagset
nltk.download('brown')
nltk.download('universal_tagset')

# Hàm tính toán xác suất và tạo các bảng cần thiết cho HMM
def train_hmm(tagged_sentences):
    tags = set(tag for sentence in tagged_sentences for _, tag in sentence)
    vocab = set(word for sentence in tagged_sentences for word, _ in sentence)

    # Tạo index cho từ và nhãn
    word_to_index = {word: i for i, word in enumerate(vocab)}
    tag_to_index = {tag: i for i, tag in enumerate(tags)}

    # Khởi tạo các ma trận xác suất
    start_prob = np.zeros(len(tags))
    transition_prob = np.zeros((len(tags), len(tags)))
    emission_prob = np.zeros((len(tags), len(vocab)))

    # Tính toán xác suất
    for sentence in tagged_sentences:
        for i, (word, tag) in enumerate(sentence):
            word_idx = word_to_index[word]
            tag_idx = tag_to_index[tag]
            if i == 0:
                start_prob[tag_idx] += 1
            else:
                prev_tag_idx = tag_to_index[sentence[i-1][1]]
                transition_prob[prev_tag_idx, tag_idx] += 1
            emission_prob[tag_idx, word_idx] += 1

    # Chuẩn hóa các ma trận xác suất
    start_prob /= start_prob.sum()
    transition_prob /= transition_prob.sum(axis=1, keepdims=True)
    emission_prob /= emission_prob.sum(axis=1, keepdims=True)

    return start_prob, transition_prob, emission_prob, word_to_index, tag_to_index, tags, vocab

# Hàm giải thuật Viterbi
def viterbi(observation, start_prob, transition_prob, emission_prob, word_to_index, tag_to_index, tags):
    observation_indices = [word_to_index[word] for word in observation]
    n_states = len(tags)
    T = len(observation)

    viterbi_matrix = np.zeros((n_states, T))
    backpointer_matrix = np.zeros((n_states, T), dtype=int)

    # Khởi tạo
    viterbi_matrix[:,0] = start_prob * emission_prob[:,observation_indices[0]]
    backpointer_matrix[:,0] = 0

    # Quét qua các từ trong câu
    for t in range(1, T):
        for s in range(n_states):
            trans_prob = viterbi_matrix[:,t-1] * transition_prob[:,s]
            max_trans_prob = np.max(trans_prob)
            viterbi_matrix[s,t] = max_trans_prob * emission_prob[s,observation_indices[t]]
            backpointer_matrix[s,t] = np.argmax(trans_prob)

    # Truy vết và tạo dãy nhãn
    best_path = np.zeros(T, dtype=int)
    # Truy vết lại đường đi tối ưu từ cuối về đầu
    best_path[-1] = np.argmax(viterbi_matrix[:, T-1])
    for t in range(T-2, -1, -1):
        best_path[t] = backpointer_matrix[best_path[t+1], t+1]

    # Chuyển đổi index về nhãn
    index_to_tag = {index: tag for tag, index in tag_to_index.items()}
    best_tags = [index_to_tag[index] for index in best_path]

    return best_tags


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [ ]:
# Lấy dữ liệu và huấn luyện mô hình
tagged_sentences = brown.tagged_sents(tagset='universal')
print(len(tagged_sentences))
tagged_sentences = tagged_sentences[:10000]
start_prob, transition_prob, emission_prob, word_to_index, tag_to_index, tags, vocab = train_hmm(tagged_sentences)

57340


In [ ]:
# Câu cần gán nhãn từ loại
observation_sentence = "The market is open every Sunday".lower().split()

# Kiểm tra xem từ nào không có trong từ điển và thêm nó vào
for word in observation_sentence:
    if word not in word_to_index:
        # Thêm từ không xác định vào từ điển với xác suất phát sinh nhỏ
        n = len(word_to_index)
        word_to_index[word] = n
        new_emission_prob = np.zeros((len(tags), len(word_to_index)))
        new_emission_prob[:,:-1] = emission_prob
        new_emission_prob[:,-1] = 1e-6
        emission_prob = new_emission_prob

# Áp dụng giải thuật Viterbi để gán nhãn
predicted_tags = viterbi(observation_sentence, start_prob, transition_prob, emission_prob, word_to_index, tag_to_index, tags)

# In kết quả
for word, tag in zip(observation_sentence, predicted_tags):
    print(f"{word}/{tag}")


the/DET
market/NOUN
is/VERB
open/VERB
every/DET
sunday/NOUN
